<a href="https://colab.research.google.com/github/supreeth-ratam/Assignment_yun/blob/main/Task_2_yun_solutions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import re

In [105]:
from PyPDF2 import PdfReader
import requests
import io
from bs4 import BeautifulSoup

In [110]:
import datefinder

Read in the Excel file (attached - corpo_announcement) using the pandas' library.
- Extract the relevant columns(like, HEADLINE, NEWSSUB, MORE) containing the earnings call dates, URLs, and any other relevant information. (If required info is not found in the text then read the pdf which is present in the "SOURCE" column and extract info from that.)
- Use regular expressions or other methods to identify the links to the earnings call transcripts and audio files within the URL column.
- Output a new Excel file containing the relevant data, along with columns containing the links to the transcripts and audio files.
- Note that you may need to use web scraping techniques to actually download the transcripts and audio files once you have identified their URLs. Be sure to properly handle any errors or exceptions that may occur during the data extraction process.

In [77]:
df = pd.read_excel('./corpo_announcements.xlsx')

In [78]:
data = df[['ATTACHMENTNAME','HEADLINE','NEWSSUB','MORE','SOURCE']].copy()

In [ ]:
df.columns

Index(['ATTACHMENTNAME', 'HEADLINE', 'NEWSID', 'NEWSSUB', '_company',
       'AGENDA_ID', 'ANNOUNCEMENT_TYPE', 'CATEGORYNAME', 'CRITICALNEWS',
       'DT_TM', 'FILESTATUS', 'Fld_Attachsize', 'MORE', 'NEWS_DT', 'NSURL',
       'OLD', 'PDFFLAG', 'QUARTER_ID', 'RN', 'SCRIP_CD', 'SLONGNAME',
       'TimeDiff', 'TotalPageCnt', 'XML_NAME', 'disseminated_time',
       'last_update', 'received_time', 'SOURCE'],
      dtype='object')

In [ ]:
df['PDFFLAG'].value_counts()

0    29268
Name: PDFFLAG, dtype: int64

In [ ]:
df['CATEGORYNAME'].value_counts()

Company Update            16967
Insider Trading / SAST     2178
Result                     1504
Board Meeting              1378
AGM/EGM                     910
 Corp Action                434
Corp. Action                382
Corp Action                   1
Name: CATEGORYNAME, dtype: int64

## Extraxting pdf's and mp3's urls

In [81]:
def extract_pdf_links(text):
    pdf_pattern = r"\bhttps://.*\.pdf\b"
    pdf_links = re.findall(pdf_pattern, text)
    if pdf_links:
        return pdf_links[0]
        print(pdf_links)
    else:
        return None

In [82]:
def extract_mp3_links(text):
    mp3_pattern = r"\bhttps://.*\.mp3\b"
    mp3_links = re.findall(mp3_pattern, text)
    if mp3_links:
        return mp3_links
        print(mp3_links)[0]
    else:
        return None

In [119]:
def extract_dates(text):
  matches = datefinder.find_dates(text)
  if matches:
    return ','.join([str(match) for match in matches])
  else:
    return None

In [93]:
def counter(pdfs):
  count = 0
  for item in pdfs:
    if (item != None):
      count+=1
  return count

In [120]:
pdfs = []
mp3s = []
dates = []

for item in data['HEADLINE']:
  pdfs.append(extract_pdf_links(str(item)))
  mp3s.append(extract_mp3_links(str(item)))
  dates.append(extract_dates(str(item)))

for idx,item in enumerate(data['NEWSSUB']):
  if pdfs[idx] == None: 
    pdfs[idx] = extract_pdf_links(str(item))
  if mp3s[idx] == None: 
    mp3s[idx] = extract_mp3_links(str(item))
  if dates[idx] == None: 
    dates[idx] = extract_dates(str(item))


for idx,item in enumerate(data['MORE']):
  if pdfs[idx] == None:
    pdfs[idx] = extract_pdf_links(str(item))
  if mp3s[idx] == None: 
    mp3s[idx] = extract_mp3_links(str(item))
  if dates[idx] == None: 
    dates[idx] = extract_dates(str(item))

print(counter(pdfs),counter(mp3s),counter(dates)) #Number of pdf's in SubNews + headlines + more

/usr/local/lib/python3.10/dist-packages/dateutil/parser/_parser.py:1207: UnknownTimezoneWarning: tzname M identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "
/usr/local/lib/python3.10/dist-packages/dateutil/parser/_parser.py:1207: UnknownTimezoneWarning: tzname IST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "
/usr/local/lib/python3.10/dist-packages/dateutil/parser/_parser.py:1207: UnknownTimezoneWarning: tzname EST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not 

40 42 29268


In [123]:
data['PDF_URLS'] = pd.DataFrame(pdfs)
data['AUDIO_URLS'] = pd.DataFrame(mp3s)
data['DATES_EXTRACTED'] = pd.DataFrame(dates)

In [124]:
data.head()

,ATTACHMENTNAME,HEADLINE,NEWSSUB,MORE,SOURCE,PDF_URLS,AUDIO_URLS,DATES_EXTRACTED
0,9982e28d-ec20-4a40-a743-81f2d6d2eb86.pdf,Issue of Duplicate Share Certificates(s),PUNJAB NATIONAL BANK - 532461 - Compliances-Re...,NaN,https://www.bseindia.com/xml-data/corpfiling/A...,None,None,
1,f7b9cd02-7cac-4a0d-9989-0e8457929015.pdf,This is to inform you that pursuant to Regulat...,BALKRISHNA INDUSTRIES LTD.-$ - 502355 - Compli...,This is to inform you that pursuant to Regulat...,https://www.bseindia.com/xml-data/corpfiling/A...,None,None,"2057-05-05 00:00:00,2021-10-28 00:00:00,2022-1..."
2,6521d50d-c0f4-495d-8c9a-02e72c2dba63.pdf,Indian Energy Exchange Limited has informed th...,Indian Energy Exchange Ltd - 540750 - Closure ...,NaN,https://www.bseindia.com/xml-data/corpfiling/A...,None,None,
3,8f71f319-94c8-4c97-965b-e806299e72a7.pdf,Intimation of loss of share certificate and st...,ELGI EQUIPMENTS LTD. - 522074 - Compliances-Re...,NaN,https://www.bseindia.com/xml-data/corpfiling/A...,None,None,
4,ea1f74b1-3cb7-4a09-a5cd-7749354013d4.pdf,Pursuant to Reg 30 read with Schedule III of S...,Bharat Dynamics Ltd - 541143 - Shareholder Mee...,NaN,https://www.bseindia.com/xml-data/corpfiling/A...,None,None,"2023-05-30 00:00:00,2052-05-05 00:00:00,2022-0..."


## Adding data into a new excel sheet called output.xlsx

In [126]:
data.to_excel('output.xlsx')

## Extracting data form PDF's

In [106]:
def info(pdf_path):

	# used get method to get the pdf file
	response = requests.get(pdf_path)

	# response.content generate binary code for
	# string function
	with io.BytesIO(response.content) as f:

		# initialized the pdf
		pdf = PdfReader(f)

		# all info about pdf
		information = pdf.getDocumentInfo()
		number_of_pages = pdf.getNumPages()

	txt = f"""
	Information about {pdf_path}:
	
	Author: {information.author}
	Creator: {information.creator}
	Producer: {information.producer}
	Subject: {information.subject}
	Title: {information.title}
	Number of pages: {number_of_pages}
	"""
	print(txt)
	
	return information


In [125]:
text = info('https://www.bseindia.com/xml-data/corpfiling/AttachLive/f7b9cd02-7cac-4a0d-9989-0e8457929015.pdf')

PdfReadError: ignored